In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [155]:
# Параметры парсера
vacancy_date = []
    
vacancy =  input('Введите название вакансии: ')
params = {
        'text': vacancy, \
        'search_field': 'name', \
        'items_on_page': '20', \
        'page': ''
    }    
headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
    }
link = 'https://hh.ru/search/vacancy'
soup = requests.get(link, params=params, headers=headers)
parsed_html = bs(soup.text,'html.parser')

Введите название вакансии: puthon


In [164]:
# Кол-во страниц по запросу вкансий
pages = parsed_html.find_all('div', {'class': 'pager'})
if not pages:
    last_page_number = 1
else:
    last_page_number = int(parsed_html.find_all('a',{'data-qa':'pager-page'})[-1].text)
last_page_number

62

In [165]:
df_data_vacancies = pd.DataFrame()

for _ in range(last_page_number):

    parsed_html = bs(html.text, 'html.parser')
    vacancy_items = parsed_html.find_all('div', {'class': 'vacancy-serp-item'})
    for vacancy in vacancy_items:
    
        # наименование вакансии
        vacancy_date = {}
        vacancy_name = vacancy.find('a', {'data-qa': "vacancy-serp__vacancy-title"})\
                                    .getText()
        vacancy_date['name'] = vacancy_name

        # наименование компании
        company_name = vacancy.find('div', {'class': 'vacancy-serp-item__meta-info-company'}) \
                            .find('a') \
                            .getText()

        vacancy_date['company_name'] = company_name

        # город
        city = vacancy.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}) \
                    .getText() \
                    .split(', ')[0]

        vacancy_date['city'] = city

        # заработная плата
        salary = vacancy.find('span', {'data-qa': "vacancy-serp__vacancy-compensation"})
        if not salary:
            salary_min = None
            salary_max = None
            salary_currency = None
        else:
            salary = salary.getText() \
                            .replace(u'\xa0', u'')
            salary = re.split(r'\s|-', salary)
            if salary[0] == 'до':
                salary_min = None
                salary_max = int(''.join(salary[1:3]))
                salary_currency = salary[3]
            elif salary[0] == 'от':
                salary_min = int(''.join(salary[1:3]))
                salary_max = None
                salary_currency = salary[3]
            else:
                salary_min = int(''.join(salary[0:2]))
                salary_max = int(''.join(salary[3:5]))
                salary_currency = salary[5]

        vacancy_date['salary_min'] = salary_min
        vacancy_date['salary_max'] = salary_max
        vacancy_date['salary_currency'] = salary_currency

        # ссылка на вакансию
        vacancy_anchor = vacancy.find('span', {'class': 'resume-search-item__name'})
        vacancy_link = vacancy_anchor.find('a', {'class': 'bloko-link'})['href']

        vacancy_date['vacancy_link'] = vacancy_link
        df_data_vacancies = df_data_vacancies.append(vacancy_date, ignore_index=True)

In [166]:
df_data_vacancies

,city,company_name,name,salary_currency,salary_max,salary_min,vacancy_link
0,Нур-Султан,ТОО Business and Technology Services,Разработчик Python,None,None,None,https://spb.hh.ru/vacancy/66082286?from=vacanc...
1,Москва,ООО HeadHunter:: IT,"Data Analyst \ Аналитик данных (BI, SQL, Pytho...",руб.,210000,170000,https://spb.hh.ru/vacancy/55340822?from=vacanc...
2,Ташкент,ООО AKFA Group,Backend Developer Python/Django (Middle+),None,None,None,https://spb.hh.ru/vacancy/55859298?from=vacanc...
3,Москва,Точка,Python разработчик,руб.,450000,None,https://spb.hh.ru/vacancy/54939886?from=vacanc...
4,Казань,ООО РеРиКом,Junior Python разработчик,руб.,100000,40000,https://spb.hh.ru/vacancy/55167412?from=vacanc...
...,...,...,...,...,...,...,...
1235,Москва,ООО АСД Технолоджиз,Python Developer (Remote),руб.,350000,250000,https://spb.hh.ru/vacancy/66080516?from=vacanc...
1236,Санкт-Петербург,ООО Вайт Код,Python developer (Middle),руб.,360000,300000,https://spb.hh.ru/vacancy/54880229?from=vacanc...
1237,Екатеринбург,ООО Мелдана,Программист-разработчик Python,руб.,None,150000,https://spb.hh.ru/vacancy/55845528?from=vacanc...
1238,Казань,ООО АТРАКС ТРЕЙД,Python разработчик,руб.,None,100000,https://spb.hh.ru/vacancy/55167692?from=vacanc...
